In this tutorial, we will analyze Lipoma dataset from the [WORC Database](https://github.com/MStarmans91/WORCDatabase/tree/development).

More details on the dataset as well as the original analysis performed by their authors can be found here:

`Starmans, M. P. A. et al. (2021). The WORC* database: MRI and CT scans, segmentations, and clinical labels for 932 patients from six radiomics studies. Submitted, preprint available from https://doi.org/10.1101/2021.08.19.21262238`

`The experiments are described in the following paper: Starmans, M. P. A. et al. (2021). Reproducible radiomics through automated machine learning validated on twelve clinical applications. Submitted, preprint available from https://arxiv.org/abs/2108.08618.`

In [ ]:
# In case you haven't already installed AutoRadiomics
!pip install autorad

In [ ]:
from autorad.external.download_WORC import download_WORCDatabase
from autorad.config import config
from pathlib import Path
import pandas as pd

# Set where we will save our data and results
base_dir = Path(config.TEST_DATA_DIR)
data_dir = base_dir / "worc_data"
result_dir = base_dir / "worc_results"

%load_ext autoreload
%autoreload 2


download_WORCDatabase(
    dataset="Liver",
    data_folder=data_dir,
    n_subjects=30,
    )

In [3]:
!ls $data_dir

Liver-006  Liver-030  Liver-038  Liver-107  Liver-146  Liver-166  labels.csv
Liver-008  Liver-031  Liver-054  Liver-115  Liver-147  Liver-168
Liver-011  Liver-033  Liver-067  Liver-122  Liver-154  Liver-180
Liver-021  Liver-035  Liver-079  Liver-124  Liver-156  Liver-184
Liver-023  Liver-037  Liver-106  Liver-128  Liver-159  Liver-185


In [4]:
from autorad.data.utils import get_paths_with_separate_folder_per_case

paths_df = get_paths_with_separate_folder_per_case(data_dir, relative=True)
paths_df

,ID,image_path,segmentation_path
0,Liver-035,Liver-035/image.nii.gz,Liver-035/segmentation.nii.gz
1,Liver-067,Liver-067/image.nii.gz,Liver-067/segmentation.nii.gz
2,Liver-033,Liver-033/image.nii.gz,Liver-033/segmentation.nii.gz
3,Liver-146,Liver-146/image.nii.gz,Liver-146/segmentation.nii.gz
4,Liver-184,Liver-184/image.nii.gz,Liver-184/segmentation.nii.gz
5,Liver-124,Liver-124/image.nii.gz,Liver-124/segmentation.nii.gz
6,Liver-115,Liver-115/image.nii.gz,Liver-115/segmentation.nii.gz
7,Liver-185,Liver-185/image.nii.gz,Liver-185/segmentation.nii.gz
8,Liver-147,Liver-147/image.nii.gz,Liver-147/segmentation.nii.gz
9,Liver-122,Liver-122/image.nii.gz,Liver-122/segmentation.nii.gz


In [5]:
from autorad.data.dataset import ImageDataset
from autorad.feature_extraction.extractor import FeatureExtractor

image_dataset = ImageDataset(
    paths_df,
    ID_colname="ID",
    root_dir=data_dir,
)
extractor = FeatureExtractor(image_dataset)
feature_df = extractor.run()

[06/01/22 11:43:04] INFO     Using extraction params from /Users/p.woznicki/g ]8;id=873367;file:///Users/p.woznicki/git/AutoRadiomics/autorad/feature_extraction/extractor.py\extractor.py]8;;\:]8;id=595993;file:///Users/p.woznicki/git/AutoRadiomics/autorad/feature_extraction/extractor.py#45\45]8;;\
                             it/AutoRadiomics/autorad/config/pyradiomics_para                
                             ms/Baessler_CT.yaml                                             

                    INFO     validation.valid                                     ]8;id=692296;file:///Users/p.woznicki/git/AutoRadiomics/.venv/lib/python3.10/site-packages/pykwalify/core.py\core.py]8;;\:]8;id=643900;file:///Users/p.woznicki/git/AutoRadiomics/.venv/lib/python3.10/site-packages/pykwalify/core.py#188\188]8;;\

                    INFO     Initialized extractor pyradiomics                ]8;id=825490;file:///Users/p.woznicki/git/AutoRadiomics/autorad/feature_extraction/extractor.py\extractor.py]8;;\:]8;id=561571;file:///Users/p.woznicki/git/AutoRadiomics/autorad/feature_extraction/extractor.py#96\96]8;;\

                    INFO     Extracting features                              ]8;id=660760;file:///Users/p.woznicki/git/AutoRadiomics/autorad/feature_extraction/extractor.py\extractor.py]8;;\:]8;id=193305;file:///Users/p.woznicki/git/AutoRadiomics/autorad/feature_extraction/extractor.py#67\67]8;;\

30it [00:31,  1.06s/it]


[06/01/22 11:43:36] INFO     get_features took 31.8606059551239sec                ]8;id=714898;file:///Users/p.woznicki/git/AutoRadiomics/autorad/utils/utils.py\utils.py]8;;\:]8;id=253608;file:///Users/p.woznicki/git/AutoRadiomics/autorad/utils/utils.py#21\21]8;;\

,ID,image_path,segmentation_path,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,...,wavelet-LLL_gldm_LargeDependenceLowGrayLevelEmphasis,wavelet-LLL_gldm_LowGrayLevelEmphasis,wavelet-LLL_gldm_SmallDependenceEmphasis,wavelet-LLL_gldm_SmallDependenceHighGrayLevelEmphasis,wavelet-LLL_gldm_SmallDependenceLowGrayLevelEmphasis,wavelet-LLL_ngtdm_Busyness,wavelet-LLL_ngtdm_Coarseness,wavelet-LLL_ngtdm_Complexity,wavelet-LLL_ngtdm_Contrast,wavelet-LLL_ngtdm_Strength
0,Liver-035,/Users/p.woznicki/git/AutoRadiomics/tests/test...,/Users/p.woznicki/git/AutoRadiomics/tests/test...,v3.0.1,1.22.1,2.1.1.2,1.2.0,3.10.4,"{'minimumROIDimensions': 1, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [3.0, 5.0]},...",...,0.49033556571131404,0.009239256309735483,0.10497863178878879,17.108519817338614,0.0019064162301314488,2.195837937351879,0.0010111456662735665,426.9975349731685,0.06190203145328447,0.284974468309886
1,Liver-067,/Users/p.woznicki/git/AutoRadiomics/tests/test...,/Users/p.woznicki/git/AutoRadiomics/tests/test...,v3.0.1,1.22.1,2.1.1.2,1.2.0,3.10.4,"{'minimumROIDimensions': 1, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [3.0, 5.0]},...",...,0.1726950112519562,0.008437808020802488,0.23877842444043665,83.97021323093149,0.0023728396471364493,0.7591978668706331,0.001202498835943038,4314.0961725974375,0.053087108883254575,3.4809930821120516
2,Liver-033,/Users/p.woznicki/git/AutoRadiomics/tests/test...,/Users/p.woznicki/git/AutoRadiomics/tests/test...,v3.0.1,1.22.1,2.1.1.2,1.2.0,3.10.4,"{'minimumROIDimensions': 1, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [3.0, 5.0]},...",...,0.00238199002485883,0.0012766642609044869,0.5654204077815189,1949.3340316747724,0.0011009446497239996,0.026605131203857553,0.007636394062061128,17684.10707042901,0.37450091558254367,22.73438458589251
3,Liver-146,/Users/p.woznicki/git/AutoRadiomics/tests/test...,/Users/p.woznicki/git/AutoRadiomics/tests/test...,v3.0.1,1.22.1,2.1.1.2,1.2.0,3.10.4,"{'minimumROIDimensions': 1, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [3.0, 5.0]},...",...,0.5235105253912582,0.014409860723565572,0.10172864195906982,9.821812246039771,0.002460816641434821,1.9618138907602145,0.0021490398335429067,178.82845259100122,0.03313849502581368,0.3598787334834973
4,Liver-184,/Users/p.woznicki/git/AutoRadiomics/tests/test...,/Users/p.woznicki/git/AutoRadiomics/tests/test...,v3.0.1,1.22.1,2.1.1.2,1.2.0,3.10.4,"{'minimumROIDimensions': 1, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [3.0, 5.0]},...",...,0.3036898583747069,0.00633021869706103,0.107550038020076,33.0708290664619,0.001481358860818306,0.8257035290466124,0.0016166914766237606,829.755720150696,0.045681180646125495,1.1242349494954111


In [ ]:
feature_df.head()

In [6]:
label_df = pd.read_csv(data_dir / "labels.csv")
label_df.head()

,patient_ID,diagnosis
0,Liver-185,1
1,Liver-166,0
2,Liver-180,0
3,Liver-067,0
4,Liver-035,0


In [7]:
from autorad.data.dataset import FeatureDataset

merged_feature_df = feature_df.merge(label_df, left_on="ID",
    right_on="patient_ID", how="left")
feature_dataset = FeatureDataset(
    merged_feature_df,
    target="diagnosis",
    ID_colname="ID"
)

Split the data into training/validation/test sets:

In [8]:
splits_path = result_dir / "splits.json"
feature_dataset.split(method="train_val_test", save_path=splits_path)

{'split_type': '60% train + 20% validation + 20% test',
 'train': ['Liver-122',
  'Liver-124',
  'Liver-185',
  'Liver-023',
  'Liver-156',
  'Liver-054',
  'Liver-079',
  'Liver-128',
  'Liver-037',
  'Liver-115',
  'Liver-033',
  'Liver-106',
  'Liver-147',
  'Liver-038',
  'Liver-035',
  'Liver-067',
  'Liver-031',
  'Liver-168'],
 'val': ['Liver-006',
  'Liver-184',
  'Liver-180',
  'Liver-030',
  'Liver-011',
  'Liver-021'],
 'test': ['Liver-159',
  'Liver-008',
  'Liver-146',
  'Liver-166',
  'Liver-154',
  'Liver-107']}

In [9]:
from autorad.training.trainer import Trainer
from autorad.models.classifier import MLClassifier

models = MLClassifier.initialize_default_sklearn_models()
print(models)

[Random Forest, Logistic Regression, SVM, XGBoost]


In [11]:
trainer = Trainer(
    dataset=feature_dataset,
    models=models,
    result_dir=result_dir,
    experiment_name="Liver_detection",
)
trainer.run_auto_preprocessing(oversampling=False)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/Users/p.woznicki/git/AutoRadiomics/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.072e-03, tolerance: 3.500e-04
  model = cd_fast.enet_coordinate_descent(


[CV 1/5] END ......................alpha=1e-05;, score=-1.531 total time=   0.4s


/Users/p.woznicki/git/AutoRadiomics/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.094e-04, tolerance: 3.429e-04
  model = cd_fast.enet_coordinate_descent(
/Users/p.woznicki/git/AutoRadiomics/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.317e-03, tolerance: 3.500e-04
  model = cd_fast.enet_coordinate_descent(


[CV 2/5] END ......................alpha=1e-05;, score=-0.785 total time=   0.3s
[CV 3/5] END ......................alpha=1e-05;, score=-0.186 total time=   0.2s


...

/Users/p.woznicki/git/AutoRadiomics/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.983e-03, tolerance: 3.429e-04
  model = cd_fast.enet_coordinate_descent(


[CV 5/5] END .......alpha=0.002009233002565048;, score=-0.752 total time=   0.2s
[CV 1/5] END ......alpha=0.0023101297000831605;, score=-0.873 total time=   0.0s
[CV 2/5] END ......alpha=0.0023101297000831605;, score=-0.238 total time=   0.0s
[CV 3/5] END ......alpha=0.0023101297000831605;, score=-0.356 total time=   0.1s
[CV 4/5] END ......alpha=0.0023101297000831605;, score=-0.274 total time=   0.0s
[CV 5/5] END ......alpha=0.0023101297000831605;, score=-0.734 total time=   0.2s
[CV 1/5] END ......alpha=0.0026560877829466868;, score=-0.863 total time=   0.0s
[CV 2/5] END ......alpha=0.0026560877829466868;, score=-0.241 total time=   0.1s
[CV 3/5] END ......alpha=0.0026560877829466868;, score=-0.353 total time=   0.1s
[CV 4/5] END ......alpha=0.0026560877829466868;, score=-0.268 total time=   0.0s
[CV 5/5] END ......alpha=0.0026560877829466868;, score=-0.717 total time=   0.1s
[CV 1/5] END ......alpha=0.0030538555088334154;, score=-0.851 total time=   0.0s
[CV 2/5] END ......alpha=0.0

[06/01/22 11:45:28] INFO     Best params for Lasso: {'alpha': 0.1}             ]8;id=981194;file:///Users/p.woznicki/git/AutoRadiomics/autorad/feature_selection/selector.py\selector.py]8;;\:]8;id=569776;file:///Users/p.woznicki/git/AutoRadiomics/autorad/feature_selection/selector.py#93\93]8;;\

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	1015
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	1015
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	1015
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	1015
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	1015
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	1015
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	1015
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	0
Rejected: 	1015


BorutaPy finished running.

Iteration: 	9 / 100
Confirmed: 	0
Tentative: 	0
Rejected: 	1015


[06/01/22 11:45:39] ERROR    Boruta failed to select features.Taking all      ]8;id=687281;file:///Users/p.woznicki/git/AutoRadiomics/autorad/feature_selection/selector.py\selector.py]8;;\:]8;id=414624;file:///Users/p.woznicki/git/AutoRadiomics/autorad/feature_selection/selector.py#121\121]8;;\
                             features instead.                                               

In [ ]:
trainer.set_optimizer("optuna", n_trials=10)
trainer.run(auto_preprocess=True)

/Users/p.woznicki/git/AutoRadiomics/autorad/training/trainer.py:169: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlfc = MLflowCallback(
[I 2022-06-01 11:31:27,879] A new study created in memory with name: Liver_detection
[I 2022-06-01 11:31:28,463] Trial 0 finished with value: 0.5 and parameters: {'feature_selection_method': 'anova', 'oversampling_method': None, 'model': 'XGBoost', 'xgb_lambda': 0.014570271595538578, 'xgb_alpha': 0.00021300368327705253, 'colsample_bytree': 0.5136940145553204, 'xgb_subsample': 0.47454241292069554, 'xgb_booster': 'gbtree', 'xgb_max_depth': 5, 'xgb_min_child_weight': 8, 'xgb_eta': 2.883592210117951e-07, 'xgb_gamma': 2.53287670148807e-07, 'xgb_grow_policy': 'lossguide'}. Best is trial 0 with value: 0.5.
2022/06/01 11:31:28 INFO mlflow.tracking.fluent: Experiment with name 'Liver_detection' does not exist. Creating a new experiment.
[I 2022-06-01 11:31:28,868] Trial 1 finished with v

In [ ]:
from autorad.utils import io
from autorad.evaluation.evaluator import SimpleEvaluator
from autorad.training.trainer import Inferrer

best_params = io.load_json(result_dir / "best_params.json")
inferrer = Inferrer(params=best_params, result_dir=result_dir)
inferrer.fit_eval(feature_dataset, result_name="test")

results = pd.read_csv(result_dir / "test.csv")

evaluator = SimpleEvaluator(results.y_true, results.y_pred)
roc = evaluator.plot_roc_curve()
# roc.write_image(result_dir / "roc.png")
roc

In [ ]:
results

,y_true,y_pred
0,1,0.001669
1,0,0.088617
2,0,0.340564
3,0,0.001854
